In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from window.utils.tiling import split_locations_array
from window.utils.truths import windows_truth

In [ ]:
experiment = 'hnm'

filename_gfrc_windows = 'gfrc_' + experiment + '_windows.csv'

In [ ]:
def process_truths_gfrc_hnm(truths):
    truths.loc[:, 'filename'] = [strin.replace('/', '_') for strin in truths.file_loc]
    truths = windows_truth(truths)
    truths['oc'] = truths['oc'].add(1)
    truths['tru_class'] = truths['oc']
    return truths

In [ ]:
truth_file_gfrc = "/data/old_home_dir/ChrissyF/GFRC/yolo_valid_GFRC_bboxes.csv"
gfrc_truth = pd.read_csv(truth_file_gfrc)
gfrc_truth = process_truths_gfrc_hnm(gfrc_truth)
gfrc_truth

In [ ]:
valid_whole_image_dir_gfrc = "/data/old_home_dir/ChrissyF/GFRC/Valid/whole_images_all/"

file_dir = "/home/cmf21/pytorch_save/output_for_draft/"

gfrc_windows = pd.read_csv(file_dir + filename_gfrc_windows)
gfrc_windows

In [ ]:
# remove class one which is things that look like animals but aren't
gfrc_windows = gfrc_windows[gfrc_windows['class'] == 0]
gfrc_windows

In [ ]:
def nms_per_im(boxes_in, thresh, method='mean'):
    
    boxes_in = boxes_in.sort_values(by='conf', ascending=False)

    xmins = boxes_in.xmn
    xmaxs = boxes_in.xmx
    ymins = boxes_in.ymn
    ymaxs = boxes_in.ymx
    confs = boxes_in.conf
    clazs = boxes_in['class']

    boxes_ot = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'pred_class'])

    xmins = np.array(xmins.tolist())
    xmaxs = np.array(xmaxs.tolist())
    ymins = np.array(ymins.tolist())
    ymaxs = np.array(ymaxs.tolist())
    confs = np.array(confs.tolist())
    clazs = np.array(clazs.tolist())

    while len(xmins) > 0:

        xmn = xmins[0]
        xmns = np.array(xmins[1:])
        xmx = xmaxs[0]
        xmxs = np.array(xmaxs[1:])
        ymn = ymins[0]
        ymns = np.array(ymins[1:])
        ymx = ymaxs[0]
        ymxs = np.array(ymaxs[1:])
        cnf = confs[0]
        cnfs = np.array(confs[1:])
        clz = clazs[0]
        clzs = np.array(clazs[1:])

        ol_wid = np.minimum(xmx, xmxs) - np.maximum(xmn, xmns)
        ol_hei = np.minimum(ymx, ymxs) - np.maximum(ymn, ymns)

        ol_x = np.maximum(0, ol_wid)
        ol_y = np.maximum(0, ol_hei)

        distx = np.subtract(xmxs, xmns)
        disty = np.subtract(ymxs, ymns)
        bxx = xmx - xmn
        bxy = ymx - ymn

        ol_area = np.multiply(ol_x, ol_y)
        bx_area = bxx * bxy
        bxs_area = np.multiply(distx, disty)

        ious = np.divide(ol_area, np.subtract(np.add(bxs_area, bx_area), ol_area))
        mask_bxs = np.greater(ious, thresh)

        if np.sum(mask_bxs) > 0:
            box_ot = pd.DataFrame(index=range(1), columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'pred_class'])

            xmns = xmns[mask_bxs]
            xmxs = xmxs[mask_bxs]
            ymns = ymns[mask_bxs]
            ymxs = ymxs[mask_bxs]
            cnfs = cnfs[mask_bxs]

            if method == 'mean':
                box_ot.loc[0, 'xmn'] = np.array(np.mean(xmns), dtype=int)
                box_ot.loc[0, 'ymn'] = np.array(np.mean(ymns), dtype=int)
                box_ot.loc[0, 'xmx'] = np.array(np.mean(xmxs), dtype=int)
                box_ot.loc[0, 'ymx'] = np.array(np.mean(ymxs), dtype=int)
                box_ot.loc[0, 'conf'] = np.mean(cnfs)
                box_ot.loc[0, 'pred_class'] = clz
            elif method == 'first':
                box_ot.loc[0, 'xmn'] = xmns[0]
                box_ot.loc[0, 'ymn'] = ymns[0]
                box_ot.loc[0, 'xmx'] = xmxs[0]
                box_ot.loc[0, 'ymx'] = ymxs[0]
                box_ot.loc[0, 'conf'] = np.max(cnfs)
                box_ot.loc[0, 'pred_class'] = clz
            else:
                box_ot.loc[0, 'xmn'] = np.min(xmns)
                box_ot.loc[0, 'ymn'] = np.min(ymns)
                box_ot.loc[0, 'xmx'] = np.max(xmxs)
                box_ot.loc[0, 'ymx'] = np.max(ymxs)
                box_ot.loc[0, 'conf'] = np.max(cnfs)
                box_ot.loc[0, 'pred_class'] = clz

            mask_out = np.repeat(False, len(xmins))
            mask_out[0] = True
            mask_out[1:] = mask_bxs
            mask_out = np.logical_not(mask_out)

            xmins = xmins[mask_out]
            xmaxs = xmaxs[mask_out]
            ymins = ymins[mask_out]
            ymaxs = ymaxs[mask_out]
            confs = confs[mask_out]
            clazs = clazs[mask_out]
            
        else:
            box_ot = pd.DataFrame(index=range(1), columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'pred_class'])

            box_ot.loc[0, 'xmn'] = xmn
            box_ot.loc[0, 'ymn'] = ymn
            box_ot.loc[0, 'xmx'] = xmx
            box_ot.loc[0, 'ymx'] = ymx
            box_ot.loc[0, 'conf'] = cnf
            box_ot.loc[0, 'pred_class'] = clz

            mask_out = np.repeat(False, len(xmins))
            mask_out[0] = True
            mask_out = np.logical_not(mask_out)
            
            xmins = xmins[mask_out]
            xmaxs = xmaxs[mask_out]
            ymins = ymins[mask_out]
            ymaxs = ymaxs[mask_out]
            confs = confs[mask_out]
            clazs = clazs[mask_out]
            
        #box_ot = box_ot.reset_index(drop=True)
        boxes_ot = pd.concat((boxes_ot, box_ot), axis=0, sort=False)

    boxes_ot.loc[:, 'filename'] = boxes_in.filename.iloc[0]

    return boxes_ot

In [ ]:
def nms_for_yolo(windows_df, nms_thresh, method):
    images = np.unique(windows_df.filename)
    windows_all_ims = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'pred_class', 'filename'])
    for im in images:
        windows_im = windows_df[windows_df.filename == im]
        windows_im = nms_per_im(windows_im, nms_thresh, method)
        windows_all_ims.append(windows_im)
        windows_all_ims = pd.concat((windows_all_ims, windows_im), axis=0, ignore_index=True, sort=False)
    return windows_all_ims
    

In [ ]:
def intersection_over_union(box1, box2):
    # determine the (x, y)-coordinates of the intersection rectangle
    xa = max(box1.xmn, box2.xmn)
    xb = min(box1.xmx, box2.xmx)
    ya = max(box1.ymn, box2.ymn)
    yb = min(box1.ymx, box2.ymx)
    # compute the area of intersection rectangle
    inter_area = max(0, xb - xa + 1) * max(0, yb - ya + 1)
    # compute the area of both the prediction and ground-truth
    box1_area = (box1.xmx - box1.xmn + 1) * (box1.ymx - box1.ymn + 1)
    box2_area = (box2.xmx - box2.xmn + 1) * (box2.ymx - box2.ymn + 1)
    # compute the intersection over union 
    iou = inter_area / float(box1_area + box2_area - inter_area)
    # return the intersection over union value
    return iou

In [ ]:
def match_to_truth_im(detections_df, truth_df, iou_threshold):
    
    if detections_df.shape[0] > 0:
        results_out = pd.DataFrame(columns = ['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'filename', 'pred_class', 
                                              'tru_class'])
        results_per_im = detections_df[['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'filename', 'pred_class']]
        results_per_im = results_per_im.reset_index(drop=True)
        truths_per_im = truth_df.reset_index(drop=True)
        # best match stores the detection with the highest iou
        best_match = [False] * results_per_im.shape[0]
        # true match stores if the truth overlaps with any detections
        true_match = [False] * truth_df.shape[0]
        # matchz stores any matches that overlap but aren't the best overlap (not double counting TP, but not adding to FP)
        matchz = np.array([True] * results_per_im.shape[0])

        for idx, tru in truths_per_im.iterrows():
            iouz = []
            for res_idx, result in results_per_im.iterrows():
                iou = intersection_over_union(tru, result)
                iouz.append(iou)
            iou_ind = np.argmax(iouz)
            if iouz[iou_ind] > iou_threshold:
                if not best_match[iou_ind]: 
                    best_iou_res = results_per_im.iloc[iou_ind:(iou_ind+1), :]
                    best_iou_res = best_iou_res.reset_index(drop=True)
                    best_iou_res.loc[:, 'confmat'] = 'TP'
                    true_box = f'xmin: {tru.xmn}; xmax:{tru.xmx}; ymin: {tru.ymn}; ymax: {tru.ymx}'
                    best_iou_res.loc[:, 'tru_box'] = true_box
                    best_iou_res.loc[:, 'tru_class'] = tru.tru_class
                    results_out = pd.concat((results_out, best_iou_res), axis=0, ignore_index=True, sort=False)
                    best_match[iou_ind] = True
                    true_match[idx] = True
            # matchz removes any matches that overlap but are not the most overlapping
            match_mask = np.array(iouz) > iou_threshold
            matchz[match_mask] = False

        # use matchz to filter results to keep only those that don't overlap with truths
        results_per_im = results_per_im[matchz]
        results_per_im = results_per_im.reset_index(drop=True)

        if results_per_im.shape[0] > 0:
            results_per_im['confmat'] = 'FP'
            results_per_im['tru_box'] = ''
            results_per_im['tru_class'] = 0
        results_out = pd.concat((results_out, results_per_im), axis=0, ignore_index=True, sort=False)  
        true_match = np.array(true_match)
        true_match = np.logical_not(true_match)
        if np.sum(true_match) > 0:
            false_negatives = truth_df[['xmn', 'xmx', 'ymn', 'ymx', 'filename', 'tru_class']]
            false_negatives = false_negatives[true_match]
            false_negatives = false_negatives.reset_index(drop=True)
            false_negatives.loc[:, 'conf'] = 1.0
            false_negatives.loc[:, 'confmat'] = 'FN'
            false_negatives.loc[:, 'tru_box'] = ''
            false_negatives.loc[:, 'pred_class'] = 0
            results_out = pd.concat((results_out, false_negatives), axis=0, ignore_index=True, sort=False)
        results_out = results_out.reset_index(drop=True)
    else:
        results_out = truth_df.loc[:, ['xmn', 'xmx', 'ymn', 'ymx', 'filename', 'tru_class']]
        results_out.columns = ['xmn', 'xmx', 'ymn', 'ymx', 'filename', 'tru_class']
        results_out.loc[:, 'conf'] = 1
        results_out.loc[:, 'confmat'] = 'FN'
        results_out.loc[:, 'tru_box'] = ''
        results_out.loc[:, 'pred_class'] = 0

    return results_out 

In [ ]:
def match_to_truth(detections_df, truth_df, valid_whole_image_dir, iou_threshold):
    image_files_jpg = list(Path(valid_whole_image_dir).glob("*.jpg"))
    image_files_png = list(Path(valid_whole_image_dir).glob("*.png"))
    image_files = image_files_jpg + image_files_png
    image_files = [img.name for img in image_files]
    matched_results = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'filename', 'confmat', 'tru_box', 'pred_class', 'tru_class'])
    
    for im in image_files:
        detections_im = detections_df[detections_df.filename == im]
        truth_im = truth_df[truth_df.filename == im]
        if detections_im.shape[0] > 0:
            # detections and truths need to match
            if truth_im.shape[0] > 0:
                match_results_im = match_to_truth_im(detections_im, truth_im, iou_threshold)
            # detections and no truths - all detections false postive
            else:
                match_results_im = detections_im[['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'filename', 'pred_class']]
                match_results_im['confmat'] = 'FP'
                match_results_im['tru_box'] = ''
                match_results_im['tru_class'] = 0
        else:
            # no detections and truths - all truths false negatives
            if truth_im.shape[0] > 0:
                match_results_im = truth_im[['xmn', 'xmx', 'ymn', 'ymx', 'filename', 'tru_class']]
                match_results_im['conf'] = 1.0
                match_results_im['confmat'] = 'FN'
                match_results_im['tru_box'] = ''
                match_results_im['pred_class'] = 0
            else:
                match_results_im = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'filename', 'confmat', 'tru_box', 'pred_class', 'tru_class'])
        matched_results = pd.concat((matched_results, match_results_im), axis=0, ignore_index=True, sort=False)
    
    return matched_results

In [ ]:
def calculate_threshold_results(windows_df, truth_df, im_dir, nms_threshold=0.05, iou_threshold=0.25, method='first'):
    threshes = np.linspace(0.01, 1, 100)
    threshold_array = np.zeros((len(threshes), 8))
    image_files_jpg = list(Path(im_dir).glob("*.jpg"))
    image_files_png = list(Path(im_dir).glob("*.png"))
    image_files = image_files_jpg + image_files_png
    image_files = [img.name for img in image_files]
    for idx, th in enumerate(threshes):
        print(th)
        detections_th = windows_df[windows_df.conf > th]
        detections_th = nms_for_yolo(detections_th, nms_threshold, method)
        result_th = match_to_truth(detections_th, truth_df, im_dir, iou_threshold)
        TP = np.sum(result_th.confmat=="TP")
        FP = np.sum(result_th.confmat=="FP")
        FN = np.sum(result_th.confmat=="FN")
        RE = TP / (TP + FN)
        PR = TP / (TP + FP)
        MR = 1 - RE
        FPPI = FP / len(image_files)

        list_out = [th, TP, FP, FN, RE, PR, MR, FPPI]

        threshold_array[idx, :] = list_out 
        
    threshold_metrics = pd.DataFrame(threshold_array, columns=['threshold', 'TP', 'FP', 'FN', 'RE', 'PR', 'MR', 'FPPI'])
    return threshold_metrics
     

In [ ]:
gfrc_metrics = calculate_threshold_results(gfrc_windows, gfrc_truth, valid_whole_image_dir_gfrc, 0.25, 0.25, 'mean')

gfrc_metrics[gfrc_metrics.RE > 0.6]

In [ ]:
gfrc_metrics.to_csv(file_dir + 'gfrc_' + experiment + "_metrics.csv", index=False)

In [ ]:
gfrc_conf_threshold = 0.14

In [ ]:
def single_threshold_results(windows_df, truth_df, im_dir, thresh, nms_threshold=0.05, iou_threshold=0.25, method='first'):

    image_files_jpg = list(Path(im_dir).glob("*.jpg"))
    image_files_png = list(Path(im_dir).glob("*.png"))
    image_files = image_files_jpg + image_files_png
    image_files = [img.name for img in image_files]
    detections_th = windows_df[windows_df.conf > thresh]
    detections_th['class'] = detections_th['class'].add(1)
    detections_th = nms_for_yolo(detections_th, nms_threshold, method)
    result_th = match_to_truth(detections_th, truth_df, im_dir, iou_threshold)

    return result_th

In [ ]:
gfrc_results = single_threshold_results(gfrc_windows, gfrc_truth, valid_whole_image_dir_gfrc, gfrc_conf_threshold, 0.25, 0.25, 'mean')
gfrc_results.to_csv(file_dir + 'gfrc_' + experiment + "_results.csv", index=False)

In [ ]:
def get_image_stats(detections_df):
    unique_images = np.unique(detections_df.filename)
    tpz = []
    fpz = []
    fnz = []
    for fl in unique_images:
        results_per_im = detections_df[detections_df['filename'] == fl]
        tpz.append(np.sum(results_per_im.confmat == 'TP'))
        fpz.append(np.sum(results_per_im.confmat == 'FP'))
        fnz.append(np.sum(results_per_im.confmat == 'FN'))

    TPz = np.reshape(np.array(tpz), (len(tpz), 1))
    FPz = np.reshape(np.array(fpz), (len(fpz), 1))
    FNz = np.reshape(np.array(fnz), (len(fnz), 1))
    UIz = np.reshape(unique_images, (len(unique_images), 1))
    df_out = pd.DataFrame(np.hstack((UIz, TPz, FPz, FNz)), columns=['filename', 'TP', 'FP', 'FN'])
    
    return df_out

In [ ]:
gfrc_image_results = get_image_stats(gfrc_results)
gfrc_image_results.to_csv(file_dir + 'gfrc_' + experiment + "_image_results.csv", index=False)

In [ ]:
gfrc_image_results[gfrc_image_results.FN > 5]

In [ ]:
from pathlib import Path
from PIL import Image
import cv2
from window.utils.drawing import draw_results_on_image

def draw_res(results_all_ims, valid_whole_image_dir, image_out_dir, dataset):
    images_out = []
    fnz_out = []
    fpz_out = []

    image_files_jpg = list(Path(valid_whole_image_dir).glob("*.jpg"))
    image_files_png = list(Path(valid_whole_image_dir).glob("*.png"))
    image_files = image_files_jpg + image_files_png
    image_files = [img.name for img in image_files]

    for fl in image_files:
        # Per image
        whole_im = cv2.imread(valid_whole_image_dir + fl)
        whole_im = cv2.cvtColor(whole_im, cv2.COLOR_BGR2RGB)

        if dataset == 'gfrc':
            fl_png = fl
        else:
            fl_png = fl[:-4] + '.png'

        # calculate results
        results_per_im = results_all_ims[results_all_ims.filename == fl]

        # create list of all false negatives
        for rw in range(results_per_im.shape[0]):
            row = results_per_im.iloc[rw, :]
            if row.confmat == 'FN':
                xmn = max(0, row.xmn - 50)
                ymn = max(0, row.ymn - 50)
                xmx = min(whole_im.shape[1], row.xmx + 50)
                ymx = min(whole_im.shape[0], row.ymx + 50)
                fn_window = whole_im[row.ymn:row.ymx, row.xmn:row.xmx]
                fn_window = cv2.resize(fn_window, (fn_window.shape[1]*2, fn_window.shape[0]*2))
                fnz_out.append(fn_window)
            if row.confmat == 'FP':
                xmn = max(0, row.xmn - 50)
                ymn = max(0, row.ymn - 50)
                xmx = min(whole_im.shape[1], row.xmx + 50)
                ymx = min(whole_im.shape[0], row.ymx + 50)
                fp_window = whole_im[row.ymn:row.ymx, row.xmn:row.xmx]
                f_window = cv2.resize(fp_window, (fp_window.shape[1]*2, fp_window.shape[0]*2))
                fpz_out.append(fp_window)

        # draw results on image
        image_out = draw_results_on_image(whole_im, results_per_im)
        image_out = cv2.resize(image_out, (1840, 1228))
        images_out.append(image_out)
        image_out = cv2.cvtColor(image_out, cv2.COLOR_BGR2RGB)
        cv2.imwrite(str(image_out_dir + fl), image_out)
        
    return fnz_out, fpz_out

In [ ]:
gfrc_image_outdir = "/home/cmf21/pytorch_save/output_for_draft/" + 'gfrc_' + experiment + '_25_25/'
gfrc_fnz, gfrc_fpz = draw_res(gfrc_results, valid_whole_image_dir_gfrc, gfrc_image_outdir, 'gfrc')

In [ ]:
# FN Z138_Img02888.jpg
# FP Z247_Img14318.jpg
# TP Z124_Img13083.jpg
Image.open(gfrc_image_outdir + 'Z138_Img02888.jpg')

In [ ]:
import random
def create_mosaic(list_in, mosaic_tuple, image_size, multiplier=1, grey=False):
    rowz = mosaic_tuple[0]
    colz = mosaic_tuple[1]
    img_rw = image_size[0] 
    img_cl = image_size[1] 
    img_asp =  img_rw / img_cl
    channels = 3
    if grey:
        channels = 1
    combined_im = np.zeros((img_rw * rowz * multiplier, img_cl * colz * multiplier, channels), dtype=np.uint8)
    sample_list = random.sample(list_in, rowz*colz)
    for idx, im in enumerate(sample_list):
        im_shp = im.shape
        im_rw = im_shp[0]
        im_cl = im_shp[1]
        im_asp = im_rw / im_cl
        if im_asp > img_asp:
            tot_cls = im_shp[0] / img_asp
            ncls_to_add = int((tot_cls - im_shp[1]) / 2)
            cls_to_add = np.ones((im_shp[0], ncls_to_add, 3)) * 255
            border_im = np.hstack((cls_to_add, im, cls_to_add))
        else:
            tot_rws = im_shp[1] * img_asp
            nrws_to_add = int((tot_rws - im_shp[0]) / 2)
            rws_to_add = np.ones((nrws_to_add, im_shp[1], 3)) * 255
            border_im = np.vstack((rws_to_add, im, rws_to_add))

        im_reshape = cv2.resize(border_im, (img_cl * multiplier, img_rw * multiplier))
        col = idx % colz
        row = idx // colz
        x1 = col * img_cl * multiplier
        x2 = (col + 1) * img_cl * multiplier
        y1 = row * img_rw * multiplier
        y2 = (row + 1) * img_rw * multiplier
        combined_im[y1:y2, x1:x2, :] = im_reshape
        
    return combined_im

In [ ]:
fnz_combined = create_mosaic(gfrc_fnz, (4, 8), (60*2, 60*2))
Image.fromarray(fnz_combined)

In [ ]:
fpz_combined = create_mosaic(gfrc_fpz, (4, 8), (60*2, 60*2))
Image.fromarray(fpz_combined)

In [ ]:
outdir = "/home/cmf21/pytorch_save/output_for_draft/"
outnam_g = 'gfrc_' + experiment + '_mosaic'
fn_out_path = outdir + outnam_g + '_fn.jpg'
fp_out_path = outdir + outnam_g + '_fp.jpg'
fnz_combined = cv2.cvtColor(fnz_combined, cv2.COLOR_BGR2RGB)
cv2.imwrite(fn_out_path, fnz_combined)
fpz_combined = cv2.cvtColor(fpz_combined, cv2.COLOR_BGR2RGB)
cv2.imwrite(fp_out_path, fpz_combined)

In [ ]:
import matplotlib.pyplot as plt

def conf_mat_plot_heatmap(cm, display_labels, title_in, heatmap_type='true'):
    fig, ax = plt.subplots(figsize=(6, 4.5))
    n_classes = cm.shape[0]
    cmap = 'Greys'

    if heatmap_type == 'percent':
        sum_vals = np.sum(cm)
    elif heatmap_type == 'true':
        sum_vals = np.reshape(np.repeat(np.sum(cm, axis=1), n_classes), (n_classes, n_classes))
    elif heatmap_type == 'pred':
        sum_vals = np.reshape(np.tile(np.sum(cm, axis=0), n_classes), (n_classes, n_classes))
        print(sum_vals)

    color_mapping = np.array(np.multiply(np.divide(cm, sum_vals), 255), np.uint8)

    for i in range(n_classes):
        for j in range(n_classes):
            text_cm = format(cm[i, j], ',')
            txt_color = [1, 1, 1] if color_mapping[i, j] > 100 else [0, 0, 0]
            ax.text(j, i, text_cm, ha="center", va="center", color=txt_color, fontsize=18)
            ax.axhline(i - .5, color='black', linewidth=1.0)
            ax.axvline(j - .5, color='black', linewidth=1.0)

    ax.matshow(color_mapping, cmap=cmap)

    ax.set_xlabel("Predicted label", fontsize=16)
    ax.set_ylabel("True label", fontsize=16)
    ax.set_xticks(np.arange(n_classes))
    ax.set_yticks(np.arange(n_classes))
    ax.set_xticklabels(display_labels, fontsize=16)
    ax.set_yticklabels(display_labels, fontsize=16)
    ax.set_title(title_in, fontsize=16)
    ax.tick_params(bottom=True, labelbottom=True, top=False, labeltop=False)

    ax.set_ylim((n_classes - 0.5, -0.5))

    return ax

In [ ]:
def conf_mat_raw(true, predicted, labels):
    mat_out = np.empty((len(labels), len(labels)))
    for i, row in enumerate(labels):
        preds_row = predicted[true == row]
        for j, col in enumerate(labels):
            mat_out[i, j] = np.sum(preds_row == col)
    mat_out = np.array(mat_out, dtype=np.int)
    return mat_out

In [ ]:
def save_conf_mat_plot(cm, labels, title, results_dir, prefix):
    n_class = len(labels)
    cm_all = np.reshape(np.array(cm, dtype=np.int), (n_class, n_class))
    cm_out = conf_mat_plot_heatmap(cm_all, labels, title)
    out_path = prefix + '_confidence_matrix.png'
    results_dir = Path(results_dir)
    cm_out.get_figure().savefig(results_dir / out_path)

In [ ]:
gfrc_results['tru_class_bin'] = np.array(gfrc_results.tru_class > 0, dtype=np.int)
cm_gfrc = conf_mat_raw(gfrc_results.tru_class_bin, gfrc_results.pred_class, [0,1])

In [ ]:
outdir = "/home/cmf21/pytorch_save/output_for_draft/"
title = "GFRC binary confusion matrix \n with hard negative mining"
labels = ["none", "animal"]
affix = 'gfrc_' + experiment
save_conf_mat_plot(cm_gfrc, labels, title, outdir, affix)